## Symbolic analysis of models

In this notebook, we use symbolic mathematics to study energy-based PySB model. We derive steady-state analytical solutions to study reaction networks without costly ODE simulations. First, we load the previously developed models for RAF and RAF inhibition:

In [1]:
import toy_example_RAF_RAFi as model
from pysb.bng import generate_equations
from util_display import display_model_info

#generate the model equations 
model=model.model
generate_equations(model)

#display model informations
display_model_info(model)

Model information
Species: 6
Parameters: 12
Expressions: 8
Observables: 8
Total Rules: 2
Energy Rules: 2
Non-energy Rules: 0
Energy Patterns: 4
Reactions: 12


Next, we define the sympy systems of equations corresponding to the ODE system but with the left-hand side (the derivative definition) set to zero, meaning that the system is considered to be at steady-state: 

In [2]:
import sympy
import scipy

#create a list of expressions to be substituted into kinetic rates of reactions
species = sympy.Matrix([sympy.Symbol(f"s_{i}", nonnegative=True) for i in range(len(model.species))])
subs = {e: e.expand_expr() for e in model.expressions | model._derived_expressions}
subs.update({sympy.Symbol(f"__s{i}"): s for i, s in enumerate(species)}) 
kinetics = sympy.Matrix([r['rate'] for r in model.reactions]).xreplace(subs)    
#simplyfy kinetic
kinetics.simplify()
sm = sympy.SparseMatrix(*model.stoichiometry_matrix.shape, model.stoichiometry_matrix.todok())
obs_matrix = scipy.sparse.lil_matrix(
    (len(model.observables), len(model.species)), dtype=int
)
for i, obs in enumerate(model.observables):
    obs_matrix[i, obs.species] = obs.coefficients
om = sympy.SparseMatrix(*obs_matrix.shape, obs_matrix.todok()) 
odes = sm * kinetics
observables = om * species

The following cell currently needs to be customized to your specific model. Define conservation of mass expressions for all monomers, and an expression you would like to solve for.

In [3]:
# Define conservation of mass expressions (each equal to zero).
conservation = sympy.Matrix([
    model.parameters["R_0"] - observables[0],
    model.parameters["I_0"] - observables[1],
])

system = sympy.Matrix.vstack(odes)
# This is just R_BRAFmut_active_obs, but it could be any expression.
#R_active = sympy.Symbol('R_active')
#expression = sympy.Matrix([R_active - observables[2]])
system = sympy.Matrix.vstack(odes, conservation)
display(system)

Matrix([
[                                                              f**(1 - phi_RR)*kr_RR*s_4 - kf_RI*s_0*s_1 - 1.0*kf_RR*s_0**2 + kr_RI*s_3 + 2*kr_RR*s_2 - f**(-phi_RR)*kf_RR*s_0*s_3],
[                              f**(1 - phi_RI)*kr_RI*s_4 + 2.0*g**(1 - phi_RI)*kr_RI*s_5 - kf_RI*s_0*s_1 + kr_RI*s_3 - g**(-phi_RI)*kf_RI*s_1*s_4 - 2.0*f**(-phi_RI)*kf_RI*s_1*s_2],
[                                                                                        f**(1 - phi_RI)*kr_RI*s_4 + 0.5*kf_RR*s_0**2 - kr_RR*s_2 - 2.0*f**(-phi_RI)*kf_RI*s_1*s_2],
[                        f**(1 - phi_RR)*kr_RR*s_4 + kf_RI*s_0*s_1 - 1.0*kf_RR*s_3**2*(1/(f*g))**phi_RR - kr_RI*s_3 + 2*kr_RR*s_5*(f*g)**(1 - phi_RR) - f**(-phi_RR)*kf_RR*s_0*s_3],
[-f**(1 - phi_RI)*kr_RI*s_4 - f**(1 - phi_RR)*kr_RR*s_4 + 2.0*g**(1 - phi_RI)*kr_RI*s_5 - g**(-phi_RI)*kf_RI*s_1*s_4 + f**(-phi_RR)*kf_RR*s_0*s_3 + 2.0*f**(-phi_RI)*kf_RI*s_1*s_2],
[                                                 -2.0*g**(1 - phi_RI)*kr_RI*s_5 + 0.5

Solve the combined system of the ODEs and conservation expressions for the list of symbols used in our desired expression. There may be multiple solutions

In [4]:
#solve the symbolic systems
#solutions = sympy.solve(system, list(species), force=True, manual=True)

#unfortunately the sympy solver seems not to be able to solve even this simple system
#working on implementin a new approach tailored for mass-action kinetic systems with specific characteristics

We evaluate the expressions or observable as function of the steady state species concentrations (S_x) to calculate the amount of active RAFs:

In [5]:
# to be done